In [ ]:
try:
    import socket
    import io
    import sys
    import fcntl
    import sys
    import re
    import signal
    import time
    import psycopg2
    from psycopg2 import pool
    import RPi.GPIO as GPIO
    import configparser
    import logging
    import threading
    from datetime import datetime, timezone
    from AtlasI2C import AtlasI2C
    import csv
    from collections import deque
    import queue
    from queue import Queue
    from threading import Timer
except ImportError as e:
    logging.error(f"ImportError: {e}")
    sys.exit(1)

logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    #filename='error.log',
                    handlers=[logging.FileHandler('error.log'), logging.StreamHandler()])

# GPIO setup
GPIO_PIN = 25
PUMP_PIN = 24
PUMP_PIN_2 = 23
HEATER_PIN = 4

GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.setup(GPIO_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN, GPIO.OUT)
GPIO.setup(PUMP_PIN_2, GPIO.OUT)
GPIO.setup(HEATER_PIN, GPIO.OUT)


def connect_to_database():
    """
    This function reads the database configuration from config.ini and establishes a connection to the database.
    Returns the connection object if successful, or None if an error occurs.
    """
    config = configparser.ConfigParser()
    config.read('config.ini')

    host = config['database']['host']
    dbname = config['database']['dbname']
    user = config['database']['user']
    password = config['database']['password']
    sslmode = config['database']['sslmode']

    try:
        conn_string = f"host={host} user={user} dbname={dbname} password={password} sslmode={sslmode}"
        conn = psycopg2.connect(conn_string)
        logging.info("Database connection established")
        return conn
    except psycopg2.Error as e:
        logging.error(f"Database connection error: {e}")
        return None

def detectSensors():
    device_list = []
    dev = AtlasI2C()
    try:
        for address in dev.list_i2c_devices():
            dev.set_i2c_address(address)
            if 11 <= address <= 127:
                moduletype = "ERR"
                while moduletype == "ERR":
                    response = dev.query("I")
                    if ("HUM" in response):
                        moduletype = "HUM"
                    elif ("CO2" in response):
                        moduletype = "CO2"
                    elif ("PRS" in response):
                        moduletype = "PRS"
                    else:
                        error_message = "Error detecting moduletype of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)
                name = "UNKNOWN"
                while (name == "UNKNOWN") or (bool == False):
                    bool = False
                    response = dev.query("name,?")
                    if ("NAME," in response):
                        name = response.split(",")[1]
                        name = name[:9]
                        r = re.compile('.{3}-.{2}-.{2}')
                        if r.match(name):
                            bool = True
                    else:
                        error_message = "Error detecting name of sensor at address "+str(address)+": "+response+"\n"
                        sys.stderr.write(error_message)

                device_list.append(AtlasI2C(address = address, moduletype = moduletype, name = name))

        for device in device_list:
            print("device.address="+str(device.address)+", device.moduletype="+str(moduletype)+", device.name="+str(device.name))

    except Exception as e:
        logging.error(f"Error detecting sensors: {e}")
        return []
    return device_list

def setProperties(device_list):
    try:
        for device in device_list:
            if (device.moduletype == "HUM"):
                device.write("O,T,1")
                time.sleep(1)
                device.write("O,Dew,1")
                time.sleep(1)
            if (device.moduletype == "CO2"):
                device.write("O,t,1")
                time.sleep(1)
    except Exception as e:
        logging.error(f"Error setting properties: {e}")

def writeHeader(output_file, device_list):
    try:
        csv_header = "timestamp, hostname, sensor ID, reading Type, Value"
        output_file.write(csv_header+"\n")
    except Exception as e:
        logging.error(f"Error writing header: {e}")

def requestReading(device_list):
    for device in device_list:
        try:
            device.write("R")
            time.sleep(1)
        except:
            error_message = "Error sending request to sensor "+str(device.name)+"\n"
            sys.stderr.write(error_message)
            continue

def pollDevice(device):
    reading = ""
    errMsg = ""
    try:
        reading = ""
        errMsg = ""
        try:
            device_output = device.read()
            device_output.encode('ascii')
            device_output = re.sub(r"[^a-zA-Z0-9: ,.]", "", device_output)
        except:
            print("Reading I2C device failed: "+device.name)
            return reading, str(device_output)

        if (device.moduletype == "HUM"):
            try:
                data = device_output.split(': ')[1]
                hum = float(data.split(',')[0])
                temp = float(data.split(',')[1])
                dew = float(data.split(',')[3])
            except:
                hum = 0.0
                temp = 0.0
                dew = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+ ". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device:"
            reading += str(hum) + ", "
            reading += str(temp) + ", "
            reading += str(dew)
        if (device.moduletype == "CO2"):
            try:
                data = device_output.split(': ')[1]
                co2 = float(data.split(',')[0])
                temp = float(data.split(',')[1])
            except:
                co2 = 0.0
                temp = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(co2) + ", "
            reading += str(temp)
        if (device.moduletype == "PRS"):
            try:
                data = device_output.split(': ')[1]
                prs = data
            except:
                prs = 0.0
                error_message = "Unexpected reading on device name: "+str(device.name)+". Reading: "+str(device_output.split(': ')[1])+"\n"
                sys.stderr.write(error_message)
                errMsg = "Unexpected reading on device"
            reading += str(prs)

        return reading, str(device_output), errMsg
    except Exception as e:
        logging.error(f"Unexpected error in device poll: {e}")
        return "", "", "Unexpected error"

# CONSTANTS

HUMIDITY_THRESHOLD_MAX = 20 # % RH
HUMIDITY_THRESHOLD_MIN = 10 # % RH
TEMP_THRESHOLD = 25 # deg C
CO2_THRESHOLD = 400

co2_readings = deque(maxlen=4)
hum_readings = deque(maxlen=4)
temp_readingss = deque(maxlen=4)
temp_readings = deque(maxlen=4)


PUMP_ON_DURATION = 30  # Pump on for 30 sec
PUMP_ON_DURATION_2 = 10
HEATER_ON_DURATION = 60
WARM_UP_TIME = 30  # Warm up for 3 minutes
POLL_INTERVAL = 10  # Poll every 10 seconds

CO2_CONTROL_INTERVAL = 60  # Control CO2 every 60 seconds
HUMIDITY_CONTROL_INTERVAL = 45
TEMP_CONTROL_INTERVAL = 155

co2_queue = Queue()
hum_queue = Queue()
temp_queue = Queue()

# ~~~~
def turn_pump_off_after_duration(duration):
    timer = Timer(PUMP_ON_DURATION, turn_pump_off)
    timer.start()


# ~~~
def open_solenoid_valve():
    try:
        GPIO.output(GPIO_PIN, GPIO.HIGH)
        time.sleep(.05)
        GPIO.output(GPIO_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error opening solenoid valve: {e}")
        sys.exit(1)

def turn_pump_on():
    try:
        GPIO.output(PUMP_PIN, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION)
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump on: {e}")
        sys.exit(1)

def turn_pump_2_on():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.HIGH)
        time.sleep(PUMP_ON_DURATION_2)
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 on: {e}")
        sys.exit(1)

def turn_pump_off():
    try:
        GPIO.output(PUMP_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump off: {e}")

def turn_pump_2_off():
    try:
        GPIO.output(PUMP_PIN_2, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error turning pump 2 off: {e}")

def turn_heater_on():
    try:
        GPIO.output(HEATER_PIN, GPIO.HIGH)
        time.sleep(HEATER_ON_DURATION)
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

def turn_heater_off():
    try:
        GPIO.output(HEATER_PIN, GPIO.LOW)
    except Exception as e:
        logging.error(f"Error with heater {e}")
        sys.exit(1)

def control_co2_based_on_average(data_queue):
    co2_readings = []
    while True:
        try:
            if not co2_queue.empty():
                data_type, value = co2_queue.get()
                
                if data_type == "CO2":
                    co2_readings.append(value)
                    avg_co2_level = sum(co2_readings) / len(co2_readings)
                    print(f"Average CO2 Level: {avg_co2_level}")
                    
                    if avg_co2_level < CO2_THRESHOLD:
                        print("Opening valve")
                        open_solenoid_valve()
                        
        except Exception as e:
            logging.error(f"Error in CO2 control loop: {e}")
            
        time.sleep(CO2_CONTROL_INTERVAL)


def control_hum_based_on_average(data_queue):
    hum_readings = []
    while True:
        try:
            if not hum_queue.empty():
                data_type, value = hum_queue.get()
                
                if data_type == "Humidity":
                    hum_readings.append(value)
                    avg_hum_level = sum(hum_readings) / len(hum_readings)
                    print(f"Average Humidity Level: {avg_hum_level}")
                    
                    if avg_hum_level > HUMIDITY_THRESHOLD_MAX:
                        print("Turning on dehumidifier for")
                        turn_pump_on()
                        turn_pump_off_after_duration(PUMP_ON_DURATION)
                        #turn_pump_off()
                        
        except Exception as e:
            logging.error(f"Error in humidity control loop: {e}")
            
        time.sleep(HUMIDITY_CONTROL_INTERVAL)
        

def hum_adder(data_queue):
    hum_readings = []
    while True:
        try:
            if not hum_queue.empty():
                data_type, value = hum_queue.get()
                
                if data_type == "Humidity":
                    hum_readings.append(value)
                    avg_hum_level = sum(hum_readings) / len(hum_readings)
                    print(f"Average Humidity Level: {avg_hum_level}")
                    
                    if avg_hum_level < HUMIDITY_THRESHOLD_MIN:
        
                        print("Turning on humidifier for {PUMP_ON_DURATION_2} seconds")
                        turn_pump_2_on()
                        turn_pump_off_after_duration(PUMP_ON_DURATION_2)
                        
        except Exception as e:
            logging.error(f"Error in humidity control loop: {e}")
            
        time.sleep(HUMIDITY_CONTROL_INTERVAL)


def heater(data_queue):
    temp_readings = []
    while True:
        try:
            if not temp_queue.empty():
                data_type, value = temp_queue.get()
                
                if data_type == "Temperature":
                    temp_readings.append(value)
                    avg_temp_level = sum(temp_readings) / len(temp_readings)
                    print(f"Average Temperature: {avg_temp_level}")
                    
                    if avg_temp_level < TEMP_THRESHOLD:
                        print("Turning on heater")
                        turn_heater_on()
                        time.sleep(HEATER_ON_DURATION)
                        turn_heater_off()
                        
        except Exception as e:
            logging.error(f"Error in heater control loop: {e}")
            
        time.sleep(TEMP_CONTROL_INTERVAL)

# code for heating pad 
def get_PWM(frequency=0.1, duty_cycle=0, pin=12): # duty_cycle in [0,100]
    GPIO.setmode(GPIO.BOARD)
    GPIO.setup(pin, GPIO.OUT)
    
    p = GPIO.PWM(pin, frequency)
    p.start(duty_cycle)
    return p

def cleanup_PWM(p):
    p.stop()
    GPIO.cleanup()

def get_plate_temperature(adc):
    gain = 4096  # +/- 4.096V
    sps = 250  # 250 samples per second
    
    voltsCh1 = adc.readADCSingleEnded(1, gain, sps) / 1000
    rawCh1 = adc.readRaw(1, gain, sps) 
    # print ("Channel 1 =%.6fV raw=0x%4X dec=%d" % (voltsCh1, rawCh1, rawCh1))
    voltsCh3 = adc.readADCSingleEnded(3, gain, sps) / 1000
    rawCh3 = adc.readRaw(3, gain, sps) 
    # print ("Channel 3 =%.6fV raw=0x%4X dec=%d" % (voltsCh3, rawCh3, rawCh3))
    # print ("Ratio 1/3 =%.6f" % (voltsCh1/voltsCh3))
    
    T_ZERO = -273.15
    T_BASE = 25.0
    BETA = 3950.0
    T_BASE_K = (T_BASE - T_ZERO)
    A = (voltsCh3-voltsCh1) / voltsCh3
    L = math.log((1 - A) / A)
    T = 1.0 / (L / BETA + 1 / (T_BASE_K)) + T_ZERO
    # print ("T =%.2f" % (T))
    return T

def print_time():
    print(get_datetime_string)

def get_datetime_string():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def is_ascii(s):
    """Check if the characters in string s are in ASCII, U+0-U+7F."""
    # return len(s) == len(s.encode())
    try:
        s.encode('ascii')
        return True
    except UnicodeEncodeError:
        return False

def print_Atlas_devices(device_list, device):
    for i in device_list:
        if(i == device):
            print("--> " + i.get_device_info())
        else:
            print(" - " + i.get_device_info())
    #print("")
    
       

# ~~~~

def main():
    turn_pump_off()
    turn_heater_off()
    try:
        loop_counter = 0
        conn = connect_to_database()
        if conn is None:
            sys.exit(1)
        cursor = conn.cursor()
        device_list = detectSensors()
        if (not device_list):
            sys.stderr.write("No sensors detected. Quitting...\n")
            sys.exit(1)

        setProperties(device_list)

        output_filename = "/home/chef/Desktop/data/" + datetime.now().strftime("%Y-%m-%d_%H:%M:%S") + ".csv"
        header_db = "INSERT INTO readings (time_point, device_id, sensor_id, reading_type, reading_value, \
            error_code, probe_string) VALUES (%s, %s, %s, %s, %s, %s, %s);"
        with open(output_filename, "w", buffering=1) as output_file:
            writeHeader(output_file, device_list)
            while True:
                requestReading(device_list)
                time.sleep(1)
                for device in device_list:
                    currentReading, probeString, errorMessage = pollDevice(device)
                    data = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f ")+time.strftime("%Z")
                    date_time = datetime.now()
                    data += ", "+str(socket.gethostname()) + ", " + device.name
                    if (device.moduletype == "HUM"):
                        data1 = data + ", humidity, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", temperature," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")
                        data3 = data + ", dew point," + currentReading.split(',')[2]
                        output_file.write(str(data3)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "humidity", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "temperature", currentReading.split(',')[1], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "dew point", currentReading.split(',')[2], \
                            errorMessage, probeString))

                        hum_level = float(currentReading.split(',')[0])
                        temp_level = float(currentReading.split(',')[1])

                        if hum_level > 0:
                            hum_queue.put(("Humidity", hum_level))
                            #print(f"HUM level for {device.name}: {hum_level}")
                            #print(f"Global HUM Levels: {hum_readings}")

                        if temp_level > 0 :
                            temp_queue.put(("temperature", temp_level))
                            #temp_readingss.append(temp_level)
                            #print(f"TEMP level: {temp_level}")
                            #print(f"Global TEMP Levels: {temp_readings}")

                    if (device.moduletype == "CO2"):
                        data1 = data + ", CO2, " + currentReading.split(',')[0]
                        output_file.write(str(data1)+"\n")
                        data2 = data + ", sensor temp," + currentReading.split(',')[1]
                        output_file.write(str(data2)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "CO2", currentReading.split(',')[0], \
                            errorMessage, probeString))
                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "sensor temp", currentReading.split(',')[1], \
                            errorMessage, probeString))

                        co2_level = float(currentReading.split(',')[0])
                        if co2_level > 0:
                            co2_queue.put(("CO2", co2_level))
                            
                        #co2_level = float(currentReading.split(',')[0])
                        #if co2_level > 0:
                            #co2_readings.append(co2_level)
                            #print(f"CO2 level for {device.name}: {co2_level}")
                            #print(f"Global CO2 Levels: {co2_readings}")


                    if (device.moduletype == "PRS"):
                        data1 = data + ", pressure, " + currentReading
                        output_file.write(str(data1)+"\n")

                        cursor.execute(header_db, (date_time, str(socket.gethostname()), \
                            device.name, "pressure", currentReading, \
                            errorMessage, probeString))
                    conn.commit()
               
                try:
                    plate_temp = get_plate_temperature(adc)
                    if plate_temp > MAX_PLATE_TEMP:
                        print(f"{get_datetime_string()} Warning: Plate temperature exceeded max limit.")
                        plate_temp = MAX_PLATE_TEMP
                except Exception as e:
                    print(f"{get_datetime_string()} Error getting plate temperature: {e}")
                    plate_temp = MAX_PLATE_TEMP

                target_temp = TARGET_TEMP  # Using the constant value instead of reading from a file

                if target_temp <= 0:
                    if pwm_state != 0:
                        print(f"{get_datetime_string()} Zero or negative target temp; turning off PWM.")
                        pwm.stop()
                        pwm_state = 0
                    duty_cycle = 0
                else:
                    # Simplified temperature control logic
                    if plate_temp >= MAX_PLATE_TEMP:
                        duty_cycle = 0
                    elif plate_temp < target_temp:
                        duty_cycle = 100
                    else:
                        duty_cycle = max(0, 100 - int((plate_temp - target_temp) * 10))  # Example linear control logic

                    try:
                        if pwm_state == 0:
                            pwm.start(duty_cycle)
                            pwm.ChangeFrequency(0.1)
                            pwm_state = 1
                        else:
                            pwm.ChangeDutyCycle(duty_cycle)
                    except Exception as e:
                        print(f"{get_datetime_string()} Error setting PWM duty cycle: {e}")

                print(f"{get_datetime_string()}, {co2}, {hum}, {temp}, {plate_temp}, {target_temp}, {duty_cycle}")


if __name__ == '__main__':
    data_queue = queue.Queue()
    device_list = detectSensors()

    # Create and start the threads
    main_thread = threading.Thread(target=main)
    co2_read_thread = threading.Thread(target=control_co2_based_on_average, args=(data_queue,))
    dehum_thread = threading.Thread(target=control_hum_based_on_average, args=(data_queue,))
    hum_thread = threading.Thread(target=hum_adder, args=(data_queue,))
    temp_thread = threading.Thread(target=heater, args=(data_queue))

    main_thread.start()
    co2_read_thread.start()
    dehum_thread.start()
    hum_thread.start()
    temp_thread.start()

    main_thread.join()
    co2_read_thread.join()
    dehum_thread.join()
    hum_thread.join()
    temp_thread.join()